In [1]:
import pymongo
import json
from tqdm import tqdm

# Connect to your MongoDB instance
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["vaers"]
col = db["reports"]
target_db = client["vaers"]
target_col = target_db["combinations COVID"]

In [2]:
with open('/home/sebastian/Documents/Masterarbeit/Combinations/Covid_Combinations_Filtered.json') as f:
    data = json.load(f)

nur einträge mit min 1 COVID Impfung berücksichtigen 
De, DE, dE, de neu berechnen
Confidence Intervalle neu berechnen

In [7]:
N_query = {"vax_data.VAX_TYPE": {"$regex": "COVID"}}
N = col.count_documents(N_query)
for item in tqdm(data, desc="Processing items", unit="item"):
    
    D_query = {
        "vax_data": {
            "$elemMatch": {
                "VAX_MANU": item["manufacturer"],
                "VAX_TYPE": item["vaccine"]
            }
        }
    }
    D = col.count_documents(D_query)
    
    E_query = {
        "symptoms": {
            "$elemMatch": {
                "$in": [item["Event"]]
            }
        },
        "vax_data.VAX_TYPE": {
            "$regex": "COVID",
            "$options": "i"
            }
    }

    E = col.count_documents(E_query)
    
    DE_query = {'$and': [D_query, E_query]}
    DE = col.count_documents(DE_query)
    
    De = D - DE
    dE = E - DE
    de = N - (DE + De + dE)
    
    entry = {
        "vaccine": item["vaccine"],
        "manufacturer": item["manufacturer"],
        "symptom": item["Event"],  # Assuming the symptom key is "Event" in your data
        "DE": DE,
        "dE": dE,
        "De": De,
        "de": de
    }
    
    # Insert the entry into the MongoDB collection
    target_col.insert_one(entry)


Processing items: 100%|██████████| 25554/25554 [12:11:16<00:00,  1.72s/item]  
